In [ ]:
!pip install datasets bitsandbytes trl transformers peft huggingface-hub accelerate safetensors pandas matplotlib

In [64]:
!pip install wandb -qU

In [65]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chaubey-amit017 (hectorlabs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [77]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    #AutoPeftModelForCausalLM, # Removed from transformers
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer, SFTConfig
# from trl.trainer.utils import DataCollatorForCompletionOnlyLM
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM # Added to peft
from huggingface_hub import notebook_login
from trl import SFTTrainer, SFTConfig, setup_chat_format, DataCollatorForCompletionOnlyLM

ImportError: cannot import name 'DataCollatorForCompletionOnlyLM' from 'trl' (/usr/local/lib/python3.11/dist-packages/trl/__init__.py)

In [5]:
supported = torch.cuda.is_bf16_supported(including_emulation = False)
print(supported)

True


In [6]:
dtype_compute = torch.bfloat16 if supported else torch.float16
print(dtype_compute)

torch.bfloat16


In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = dtype_compute
)

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    "HuggingFaceTB/SmolLM3-3B", device_map = "cuda:0", quantization_config = bnb_config
)
model = prepare_model_for_kbit_training(model)

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/182 [00:00<?, ?B/s]

In [12]:
print(model)

SmolLM3ForCausalLM(
  (model): SmolLM3Model(
    (embed_tokens): Embedding(128256, 2048, padding_idx=128004)
    (layers): ModuleList(
      (0-35): 36 x SmolLM3DecoderLayer(
        (self_attn): SmolLM3Attention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): SmolLM3MLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): SmolLM3RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): SmolLM3RMSNorm((2048,), eps=1e-06)
      )
   

In [21]:
import re

def find_all_linear_names(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 3B models
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

print(find_all_linear_names(model))

['o_proj', 'gate_proj', 'v_proj', 'down_proj', 'up_proj', 'k_proj', 'q_proj']


In [31]:
config = LoraConfig(
    r = 16,
    lora_alpha = 32,
    lora_dropout = 0.05,
    target_modules = ["query_key_value", "o_proj", "qkv_proj", "gate_up_proj", "down_proj"],
    bias = "none",
    task_type = "CAUSAL_LM"
)
peft_model = get_peft_model(model, config)

In [32]:
dataset = load_dataset("sweatSmile/sarcastic-dataset", split="train")
dataset

README.md:   0%|          | 0.00/347 [00:00<?, ?B/s]

sarcastic_dataset_cleaned.csv: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['sentence', 'translation', 'translation_extra'],
    num_rows: 720
})

In [33]:
dataset = dataset.rename_column("sentence", "prompt")
dataset = dataset.rename_column("translation_extra", "completion")
dataset = dataset.remove_columns(["translation"])
dataset

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 720
})

In [37]:
messages = [
    {"role": "user", "content": dataset[0]['prompt']},
    {"role": "assistant", "content": dataset[0]['completion']}
]
messages

[{'role': 'user', 'content': 'The birch canoe slid on the smooth planks.'},
 {'role': 'assistant',
  'content': "Oh, wow, a birch canoe slid on smooth planks? Who would have thought that would happen? Next, you'll tell me water's wet and the sky is blue!"}]

In [35]:
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM3-3B")
tokenizer.chat_template

'{# ───── defaults ───── #}\n{%- if enable_thinking is not defined -%}\n{%- set enable_thinking = true -%}\n{%- endif -%}\n\n{# ───── reasoning mode ───── #}\n{%- if enable_thinking -%}\n  {%- set reasoning_mode = "/think" -%}\n{%- else -%}\n  {%- set reasoning_mode = "/no_think" -%}\n{%- endif -%}\n\n{# ───── header (system message) ───── #}\n{{- "<|im_start|>system\\n" -}}\n\n{%- if messages[0].role == "system" -%}\n  {%- set system_message = messages[0].content -%}\n  {%- if "/no_think" in system_message -%}\n    {%- set reasoning_mode = "/no_think" -%}\n  {%- elif "/think" in system_message -%}\n    {%- set reasoning_mode = "/think" -%}\n  {%- endif -%}\n  {%- set custom_instructions = system_message.replace("/no_think", "").replace("/think", "").rstrip() -%}\n{%- endif -%}\n\n{%- if "/system_override" in system_message -%}\n  {{- custom_instructions.replace("/system_override", "").rstrip() -}}\n  {{- "<|im_end|>\\n" -}}\n{%- else -%}\n  {{- "## Metadata\\n\\n" -}}\n  {{- "Knowledg

In [38]:
print(tokenizer.apply_chat_template(messages, tokenize=False))

<|im_start|>system
## Metadata

Knowledge Cutoff Date: June 2025
Today Date: 05 August 2025
Reasoning Mode: /think

## Custom Instructions

You are a helpful AI assistant named SmolLM, trained by Hugging Face. Your role as an assistant involves thoroughly exploring questions through a systematic thinking process before providing the final precise and accurate solutions. This requires engaging in a comprehensive cycle of analysis, summarizing, exploration, reassessment, reflection, backtracking, and iteration to develop well-considered thinking process. Please structure your response into two main sections: Thought and Solution using the specified format: <think> Thought section </think> Solution section. In the Thought section, detail your reasoning process in steps. Each step should include detailed considerations such as analysing questions, summarizing relevant findings, brainstorming new ideas, verifying the accuracy of the current steps, refining any errors, and revisiting previou

In [23]:
# def modify_tokenizer(tokenizer,
#                      alternative_bos_token='<|im_start|>',
#                      alternative_unk_token='<unk>',
#                      special_tokens=None,
#                      tokens=None):
#     eos_token, bos_token = tokenizer.eos_token, tokenizer.bos_token
#     pad_token, unk_token = tokenizer.pad_token, tokenizer.unk_token

#     # BOS token must be different than EOS token
#     if bos_token == eos_token:
#         bos_token = alternative_bos_token

#     # UNK token must be different than EOS token
#     if unk_token == eos_token:
#         unk_token = alternative_unk_token

#     # PAD token must be different than EOS token
#     # but can be the same as UNK token
#     if pad_token == eos_token:
#         pad_token = unk_token

#     assert bos_token != eos_token, "Please choose a different BOS token."
#     assert unk_token != eos_token, "Please choose a different UNK token."

#     # Creates dict for BOS, PAD, and UNK tokens
#     # Keeps the EOS token as it was originally defined
#     special_tokens_dict = {'bos_token': bos_token,
#                            'pad_token': pad_token,
#                            'unk_token': unk_token}

#     # If there are additional special tokens, add them
#     if special_tokens is not None:
#         if isinstance(special_tokens, list):
#             special_tokens_dict.update({'additional_special_tokens': special_tokens})

#     tokenizer.add_special_tokens(special_tokens_dict)

#     # If there are new regular (not special) tokens to add
#     if tokens is not None:
#         if isinstance(tokens, list):
#             tokenizer.add_tokens(tokens)

#     return tokenizer

In [26]:

# def jinja_template(tokenizer):
#     return ("{% for message in messages %}"
#             f"{{{{'{tokenizer.bos_token}' + message['role'] + '\n' + message['content'] + '{tokenizer.eos_token}' + '\n'}}}}"
#             "{% endfor %}"
#             "{% if add_generation_prompt %}"
#             f"{{{{ '{tokenizer.bos_token}assistant\n' }}}}"
#             "{% endif %}")


In [27]:

# def add_template(tokenizer, chat_template=None):
#     # If not chat template was given, creates a ChatML template
#     # using the BOS and EOS tokens
#     if chat_template is None:
#         chat_template = jinja_template(tokenizer)

#     # Assigns chat template to tokenizer
#     tokenizer.chat_template = chat_template

#     return tokenizer

# def get_multiple_of(vocab_size):
#     return 2**(bin(vocab_size)[::-1].find('1'))

# def modify_model(model, tokenizer):
#     # If new tokenizer length exceeds vocabulary size
#     # resizes it while keeping it a multiple of the same value
#     if len(tokenizer) > model.config.vocab_size:
#         pad_to_multiple_of = get_multiple_of(model.vocab_size)
#         model.resize_token_embeddings(len(tokenizer),
#                                       pad_to_multiple_of=pad_to_multiple_of)

#     # Updates token ids on model configurations
#     if getattr(model, "config", None) is not None:
#         model.config.pad_token_id = tokenizer.pad_token_id
#         model.config.bos_token_id = tokenizer.bos_token_id
#         model.config.eos_token_id = tokenizer.eos_token_id
#     if getattr(model, "generation_config", None) is not None:
#         model.generation_config.bos_token_id = tokenizer.bos_token_id
#         model.generation_config.eos_token_id = tokenizer.eos_token_id
#         model.generation_config.pad_token_id = tokenizer.pad_token_id

#     return model

# def generate(model, tokenizer, sentence, max_new_tokens=64, skip_special_tokens=False):
#     converted_sample = [
#         {"role": "user", "content": sentence},
#     ]
#     prompt = tokenizer.apply_chat_template(converted_sample,
#                                            tokenize=False,
#                                            add_generation_prompt=True)

#     tokenized_input = tokenizer(prompt, add_special_tokens=False, return_tensors="pt")
#     input_ids = tokenized_input["input_ids"].to(model.device)

#     model.eval()
#     generation_output = model.generate(input_ids=input_ids,
#                                        eos_token_id=tokenizer.eos_token_id,
#                                        max_new_tokens=max_new_tokens)

#     output = tokenizer.batch_decode(generation_output,
#                                     skip_special_tokens=skip_special_tokens)[0]
#     return output

In [ ]:
# peft_model = modify_model(peft_model, tokenizer)

In [67]:
tokenizer.padding_side='left'

response_template = '<|im_start|>assistant\n'
collator_fn=DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

NameError: name 'DataCollatorForCompletionOnlyLM' is not defined

#some experiments to see how the training goes, it looks like it oscillates and no sign of convergence.

In [57]:
min_effective_batch_size = 8
lr = 3e-4
# max_seq_length = 64
# collator_fn = None
# packing = (collator_fn is None)
steps = 20
num_train_epochs = 10

sft_config = SFTConfig(
    output_dir = '/content/drive/MyDrive/hf-smol3b/hf-SmolLM3-3B-ada',
    # packing = packing,
    # max_seq_length=64, # Removed from SFTConfig
    gradient_checkpointing = True,
    gradient_checkpointing_kwargs={'use_reentrant': False},
    gradient_accumulation_steps = 2,
    per_device_train_batch_size = min_effective_batch_size,
    auto_find_batch_size = True,
    num_train_epochs = num_train_epochs,
    learning_rate = lr,
    report_to = 'wandb',
    logging_dir = '/content/drive/MyDrive/hf-smol3b/hf-SmolLM3-3B/logs',
    logging_strategy = 'steps',
    logging_steps = steps,
    save_strategy = 'steps',
    save_steps = steps
)

trainer = SFTTrainer(
    model = peft_model,
    train_dataset = dataset,
    processing_class = tokenizer,
    # data_collator = collator_fn,
    args = sft_config,
    # max_seq_length = max_seq_length # Added here
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
20,3.102600
40,3.140700
60,3.116300
80,3.131800
100,3.123900
120,3.123600
140,3.099600
160,3.122500
180,3.146400
200,3.114000


KeyboardInterrupt: 

In [ ]:
def gen_prompt(tokenizer, sentence):
  pass

sentence = "What a day"
prompt = gen_prompt(tokenizer, sentence)

In [ ]:
def generate(peft_model, tokenizer, prompt, max_new_tokens=64, skip_special_tokens=True):
  tokenized_input = tokenizer(prompt,add_special_tokens=False, return_tensors="pt").to(model.device)
  model.eval()
  generation_output = model.generate(**tokenized_input, max_new_tokens=max_new_tokens, eos_token_id=tokenizer.eos_token_id)
  output = tokenizer.batch_decode(generation_output, skip_special_tokens=skip_special_tokens)[0]
  return output

In [ ]:
print(generate(peft_model, tokenizer, prompt))

#Saving the Adapter

In [ ]:
trainer.save_model('your_model_name')

#Merged the model with the unquantised base model

In [58]:
reloaded_model = AutoPeftModelForCausalLM.from_pretrained('your_model_name')
reloaded_model

NameError: name 'AutoPeftModelForCausalLM' is not defined

In [ ]:
merged_model = reloaded_model.merge_and_unload()
merged_model

In [ ]:
merged_model.save_pretrained('your_model_name')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
trainer.push_to_hub()

In [71]:
import trl
print(dir(trl))
# # # You can also try to explore submodules
# # # print(dir(trl.trainer))
# print(dir(trl.trainer.utils))

['AlignPropConfig', 'AlignPropTrainer', 'AllTrueJudge', 'AutoModelForCausalLMWithValueHead', 'AutoModelForSeq2SeqLMWithValueHead', 'BCOConfig', 'BCOTrainer', 'BaseBinaryJudge', 'BaseJudge', 'BasePairwiseJudge', 'BaseRankJudge', 'BestOfNSampler', 'CPOConfig', 'CPOTrainer', 'DDPOConfig', 'DDPOPipelineOutput', 'DDPOSchedulerOutput', 'DDPOStableDiffusionPipeline', 'DDPOTrainer', 'DPOConfig', 'DPOTrainer', 'DefaultDDPOStableDiffusionPipeline', 'FDivergenceConstants', 'FDivergenceType', 'GKDConfig', 'GKDTrainer', 'GRPOConfig', 'GRPOTrainer', 'HfPairwiseJudge', 'IterativeSFTConfig', 'IterativeSFTTrainer', 'KTOConfig', 'KTOTrainer', 'LogCompletionsCallback', 'MergeModelCallback', 'ModelConfig', 'NashMDConfig', 'NashMDTrainer', 'ORPOConfig', 'ORPOTrainer', 'OnlineDPOConfig', 'OnlineDPOTrainer', 'OpenAIPairwiseJudge', 'PPOConfig', 'PPOTrainer', 'PRMConfig', 'PRMTrainer', 'PairRMJudge', 'PreTrainedModelWrapper', 'RLOOConfig', 'RLOOTrainer', 'RewardConfig', 'RewardTrainer', 'RichProgressCallback',